In [2]:
import numpy as np
from numba import njit, prange
import threading


In [23]:
%load_ext line_profiler

In [7]:
N = 4

@njit
def make_grid(n):
    grid = np.zeros((n, n), dtype=np.int8)
    return grid

@njit
def add_sand(grid, x, y):
    grid[x, y] += 1

@njit
def add_sand_random(grid):
    n = grid.shape[0]
    x : int = np.random.randint(0, n)
    y : int = np.random.randint(0, n)
    add_sand(grid, x, y)


@njit(parallel=False)
def propagate(grid) -> bool:
    size = grid.shape[0]
    ng = np.zeros_like(grid, dtype=np.int8)


    for x in prange(size):
        for y in range(size):
            if not grid[x, y] >= N:
                continue
                
            ng[x, y] -= 4
            if (x - 1) >= 0:
                ng[x - 1, y] += 1
            elif (x + 1) < size:
                ng[x + 1, y] += 1
            if (y - 1) >= 0:
                ng[x, y - 1] += 1
            elif (y + 1) < size:
                ng[x, y + 1] += 1
    

    grid += ng
    return np.any(ng)

@njit
def topple(grid):
    while True:
        if not propagate(grid):
            break

@njit
def step(grid):
    add_sand_random(grid)
    topple(grid)
    

@njit
def run(size, n_steps):
    grid = make_grid(size)
    for i in range(n_steps):
        # print(i)
        step(grid)
    return grid



In [9]:
g = run(100, 100_000)
# %lprun -f propagate run(100, 100_0)

# from time import time
# start = time()
# run(100, 100_000)
# end = time()

# print(end - start)

KeyboardInterrupt: 

In [ ]:
%lprun -f propagate run(100, 100_000)


![Alt text](image.png)

In [31]:
not 0

True